<a href="https://colab.research.google.com/github/Santigs/DS-Course/blob/main/Lecture3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# make sure you have installed the api
# pip install pycoingecko
# https://github.com/man-c/pycoingecko
!pip install pycoingecko

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from pycoingecko import CoinGeckoAPI

In [ ]:
cg = CoinGeckoAPI()
# Getting bitcoin price: cg.get_price(ids='bitcoin', vs_currencies='usd')
# Getting list of coins: cg.get_coins_list()

coinlist = cg.get_coins_list()
coinlist


In [ ]:
# Fetching currency data
currency = 'badger-dao'
#currency = 'honey'
data = cg.get_coin_by_id(currency)

In [ ]:
# Getting current price and ath date & price
name = data['id']
market_data = data['market_data']
current_price = market_data['current_price']
all_time_high = market_data['ath']
ath_date = market_data['ath_date']
total_volume = market_data['total_volume']


price_usd = current_price['usd']
price_ath = all_time_high['usd']
date_ath = ath_date['usd']
volume_usd = total_volume['usd']

# Printing values 

print(name)
print('Current price: %3.4f USD' %price_usd)
print('Ath price: %3.4f USD' %price_ath)
print('Ath date: ',date_ath[0:10],'at', date_ath[11:19])
print('Volume: %8.2f USD' %volume_usd)

In [ ]:
# Fetch coin contract info from coingecko
# cg.get_coin_info_from_contract_address_by_id('honey')

ticker = cg.get_coin_ticker_by_id(currency)



# extract exchanges and volume
exchanges = pd.DataFrame(ticker['tickers'])
converted_volumes = exchanges['converted_volume']


# create a dataframe with volume trades per market per currency pair in usd
traded_volume = pd.DataFrame(columns=['Exchange', 'Pair', 'Amount'])
for num, volume in enumerate(converted_volumes):
    traded_volume = traded_volume.append({'Exchange': exchanges.iloc[num][2]['name'], 
                                          'Pair': exchanges.iloc[num][1], 
                                          'Amount': volume['usd']}, ignore_index=True)
    
    

    
#traded_volume.set_index('Exchange',inplace=True)
traded_volume

In [ ]:
traded_volume.info()

In [ ]:
#create a new dataframe to convert amount values from string to numeric
traded_df= pd.DataFrame(traded_volume)
traded_df['Amount']=pd.to_numeric(traded_df['Amount'])

#create a scatter chart
plt.figure(figsize=(20,10))
plt.xlabel('Exchanges', fontsize=24)
plt.ylabel('Pairs', fontsize=24)
plt.title(name+' Traded Volume', fontsize=32)
plt.xticks(rotation=45)
graph = plt.scatter(traded_df['Exchange'], traded_df['Pair'], s=traded_df['Amount']/5000, cmap='Blues')


In [ ]:
fig= plt.figure(figsize=(20,6))
graph = sns.barplot(x = 'Exchange', y = 'Amount', hue='Pair' , data = traded_df, palette = 'Greens', edgecolor = 'w')

In [ ]:
# Fetching last days of data
# https://github.com/man-c/pycoingecko/blob/master/pycoingecko/api.py

chart_data= cg.get_coin_market_chart_by_id(currency, 'usd', 30)

In [ ]:
# Generate dataframe for price and volume
data_price = pd.DataFrame(chart_data['prices'])
data_volume = pd.DataFrame(chart_data['total_volumes'])
data_price.columns = ['Date', 'Price']
data_volume.columns = ['Date', 'Volume']
data_price = data_price.join(data_volume['Volume'])


data_price['Date'] = pd.to_datetime(data_price['Date'], unit='ms')
data_price['new_formatted_date'] = data_price.Date.dt.strftime('%d/%m/%y %H:%M:%S')


data_price
#data_price.to_csv(name+'_price_data.csv', index = False)


In [ ]:
# Creating a price chart

graph1 = data_price.plot(x='Date', y='Price', figsize=(20,8), color='b')


In [ ]:
# Creating a Volume bar chart

graph2 = data_volume.plot.bar(x='Date', y='Volume', figsize=(60,10), color='b')
graph2.xaxis.set_major_formatter(plt.NullFormatter())

In [ ]:
# Fitting a polynomial curve
# to show volume trend

length = len(data_volume['Date'])


x = np.linspace(1, length, length)
y = data_volume['Volume']



p3 = np.polyfit(x, y,3)
xpoly = np.linspace(1, length, length)
ypoly = np.polyval(p3, xpoly)



graph3 = plt.figure(figsize=(60,10))
graph3 = plt.bar(x,y)
graph3 = plt.plot(xpoly, ypoly, color='r', linewidth=6)
graph3 = plt.title('Volume Evolution', fontsize=36)





In [ ]:
# Fitting a polynomial curve
# to show price trend

length = len(data_price['Date'])

x = np.linspace(1, length, length)
y = data_price['Price']


p3 = np.polyfit(x, y,3)
xpoly = np.linspace(1, length, length)
ypoly = np.polyval(p3, xpoly)

graph4 = plt.figure(figsize=(60,10))
graph4 = plt.title('Price Evolution', fontsize=36)
graph4 = plt.plot(x, y, linewidth=6)
graph4 = plt.plot(xpoly, ypoly, color='r', linewidth=4)

#labels = data_price['Date'].tolist()




### Animating a graph

In [ ]:
import random
from itertools import count
from matplotlib.animation import FuncAnimation
from matplotlib.pyplot import figure

In [ ]:
#This cell will not work on google colab
#You need to run it in your local machine with jupyter

%matplotlib notebook
import matplotlib.animation as animation
from matplotlib import rcParams


#plt.style.use('fivethirtyeight')
params = {'figure.figsize': (10, 5),
         'xtick.labelsize':'x-small',
         'ytick.labelsize':'x-small',
         }
plt.rcParams.update(params)
plt.xticks(rotation=30)

labels = []
x_vals = []
y_vals = []
y_h = data_price['Price']



index = count()

def animate(i):
    num = next(index)
    dia = data_price['new_formatted_date'][num]
    
    #labels.append(dia[1:8])
    #plt.set_xticklabels(labels)
    
    x_vals.append(data_price['Date'][num])
    y_vals.append(y_h[num])
   

    plt.cla()
    plt.xticks(rotation=30)
    #plt.xlabel('xlabel', fontsize=10)
    plt.plot(x_vals, y_vals, linewidth=2.0, color='red', label='currency')
    
    plt.legend(loc='upper left')
    plt.tight_layout
    
   


ani = FuncAnimation(plt.gcf(), animate, interval=1)


plt.plot(x_vals, y_vals)


plt.tight_layout()
plt.show()

#ani.save('myAnimation.gif', writer='imagemagick', fps=30)
#https://riptutorial.com/matplotlib/example/23577/interactive-controls-with-matplotlib-widgets
#https://www.google.com/search?q=jupyter+animate+matplotlib&oq=jupyter+animat&aqs=chrome.3.69i57j0l7.6679j0j7&sourceid=chrome&ie=UTF-8#kpvalbx=_oVGLXqPjIMiHjLsPsYqAqA820